**Table of contents**<a id='toc0_'></a>    
- [Loading Libraries](#toc1_)    
- [Extract Players'urls](#toc2_)    
- [Scraping Player Statistics](#toc3_)    
- [Data Transform](#toc4_)    
- [Teams Stats](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Loading Libraries](#toc0_)

In [1]:
import json

import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    WebDriverException
)
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc

# <a id='toc2_'></a>[Extract Players'urls](#toc0_)

In [2]:
with open('../data/raw/lineups_event13966075.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

base = "https://www.sofascore.com/es/football/player/{slug}/{id}"

def get_starting_urls(team_side):
    team = data.get(team_side, {})
    players = team.get("players", [])
    # titulares
    starters = [pl for pl in players if not pl.get("substitute", False)]
    # rellenar con suplentes si hay menos de 11
    if len(starters) < 11:
        subs = [pl for pl in players if pl.get("substitute", False)]
        starters += subs[:11 - len(starters)]
    starters = starters[:11]

    urls = []
    for pl in starters:
        player = pl.get("player", {})
        slug = player.get("slug")
        pid  = player.get("id")
        if slug and pid:
            urls.append(base.format(slug=slug, id=pid))
        else:
            print("Faltan datos para URL:", player.get("name"), slug, pid)
    return urls

home_urls = get_starting_urls("home")
away_urls = get_starting_urls("away")

# Lista única con los 11 titulares de local seguidos por los 11 titulares de visitante
urls = home_urls + away_urls

# Imprimir (o usar la lista 'urls' en tu pipeline)
for u in urls:
    print(u)

https://www.sofascore.com/es/football/player/alexis-martin-arias/340849
https://www.sofascore.com/es/football/player/fabricio-dominguez/939934
https://www.sofascore.com/es/football/player/matias-perez/989208
https://www.sofascore.com/es/football/player/quintana-lucas/1199279
https://www.sofascore.com/es/football/player/blas-riveros/805202
https://www.sofascore.com/es/football/player/wilder-viera/1000445
https://www.sofascore.com/es/football/player/jorge-morel/1175757
https://www.sofascore.com/es/football/player/juan-iturbe/146415
https://www.sofascore.com/es/football/player/rodrigo-gomez/1484114
https://www.sofascore.com/es/football/player/ignacio-aliseda/940697
https://www.sofascore.com/es/football/player/torres-jonathan/802861
https://www.sofascore.com/es/football/player/fernando-muslera/28381
https://www.sofascore.com/es/football/player/eric-meza/941092
https://www.sofascore.com/es/football/player/edwuin-cetre/927790
https://www.sofascore.com/es/football/player/facundo-rodriguez/980

# <a id='toc3_'></a>[Scraping Player Statistics](#toc0_)

In [3]:
# Lista de URLs de jugadores (ejemplo: 22 URLs)
player_urls = urls

# User‐agents variados para rotación
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36",
]


def create_stealth_driver():
    # options = uc.ChromeOptions()
    # # Rotar user-agent
    # ua = random.choice(USER_AGENTS)
    # options.add_argument(f"--user-agent={ua}")
    # # Desactivar flags de detección
    # options.add_argument("--disable-blink-features=AutomationControlled")
    # options.add_argument("--no-sandbox")
    # options.add_argument("--disable-dev-shm-usage")
    # # Ventana aleatoria
    # width = random.randint(1024, 1920)
    # height = random.randint(768, 1080)
    # options.add_argument(f"--window-size={width},{height}")

    # driver = uc.Chrome(options=options)
    # driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
    #     'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'
    # })
    # return driver

    options = uc.ChromeOptions()
    ua = random.choice(USER_AGENTS)
    options.add_argument(f"--user-agent={ua}")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    width = random.randint(1024, 1920)
    height = random.randint(768, 1080)
    options.add_argument(f"--window-size={width},{height}")

    # <— fuerza la versión 138
    driver = uc.Chrome(version_main=138, options=options)
    driver.execute_cdp_cmd(
        'Page.addScriptToEvaluateOnNewDocument',
        {'source': 'Object.defineProperty(navigator, "webdriver", {get: () => undefined})'}
    )
    return driver


def scrape_player_stats(driver, player_url):
    driver.get(player_url)
    # Espera inicial aleatoria
    time.sleep(random.uniform(2, 4))

    # Datos básicos
    try:
        name_elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h2")))
        player_name = name_elem.text.strip()
    except Exception:
        player_name = None

    try:
        team_elem = driver.find_element(By.XPATH, "(//div[contains(@class,'flex-d_column') and contains(@class,'jc_center')]//span[contains(@class,'textStyle_body')])[1]")
        team_name = team_elem.text.strip()
    except Exception:
        team_name = None

    try:
        pos_elem = driver.find_element(By.CSS_SELECTOR, "div.Box.oWZdE .Text.beCNLk")
        position = pos_elem.text.strip()
    except Exception:
        position = None

    # Click modal
    try:
        triggers = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.Box.gRFdPz")))
        trigger = triggers[-1]
        driver.execute_script("arguments[0].scrollIntoView({block:'center'})", trigger)
        time.sleep(random.uniform(0.5, 1.0))
        ActionChains(driver).move_to_element(trigger).click().perform()
    except Exception:
        pass

    # Extraer estadísticas
    stats = {}
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "span.ov-x_clip")))
        time.sleep(random.uniform(2, 3))
        # Ocultar overlays
        driver.execute_script("document.querySelectorAll('[id^=\"div-gpt-ad\"], .overlay, .banner').forEach(el => el.style.display='none');")
        labels = driver.find_elements(By.CSS_SELECTOR, "span[class*='ov-x_clip']")
        for label in labels:
            if not label.is_displayed():
                continue
            try:
                value = driver.execute_script("return arguments[0].nextElementSibling", label)
                text_val = value.text.strip() if value else ""
            except:
                continue
            ltxt = label.text.strip()
            if ltxt and text_val:
                stats[ltxt] = text_val
    except Exception:
        pass

    # Construir registro
    record = {
        "Nombre": player_name,
        "Equipo": team_name,
        "Posición": position,
        **stats
    }
    return record


def main():
    driver = create_stealth_driver()
    all_records = []
    for url in player_urls:
        try:
            rec = scrape_player_stats(driver, url)
            all_records.append(rec)
            # Espera corta antes del siguiente jugador
            time.sleep(random.uniform(1, 2))
        except Exception:
            continue

    driver.quit()

    # Convertir a DataFrame
    df = pd.DataFrame(all_records)
    print(df)
    # Guardar a CSV
    df.to_csv("jugadores_stats.csv", index=False)

    return df

if __name__ == "__main__":
    df_final = main()


                    Nombre                   Equipo Posición Minutos jugados  \
0      Alexis Martín Arias                     None        G             90'   
1       Fabricio Domínguez                     None        M             62'   
2             Matías Pérez            Cerro Porteño        D             66'   
3           Lucas Quintana                     None        D             90'   
4             Blás Riveros                     None        D             90'   
5             Wílder Viera            Cerro Porteño        M             38'   
6              Jorge Morel            Cerro Porteño        M             53'   
7              Juan Iturbe            Cerro Porteño        F             52'   
8            Rodrigo Gómez                     None        D              8'   
9          Ignacio Aliseda                     None        F             19'   
10         Jonathan Torres            Cerro Porteño        F             83'   
11        Fernando Muslera  Estudiantes 

# <a id='toc4_'></a>[Data Transform](#toc0_)

In [4]:
#Fixing team names if needed
df_final.loc[[0,1,3,4,8,9], 'Equipo'] = 'Cerro Porteño'
#df_final.loc[[], 'Equipo'] = ''
df_final.fillna(0, inplace=True)

#Minutos Jugados
df_final["Minutos jugados"] = df_final["Minutos jugados"].str.rstrip("'")

#Pases Precisos 
df_final[['Pases Acertados', 'Total Pases', 'Efectividad Pases (%)']] = df_final['Pases precisos'].str.extract(r'(\d+)/(\d+)\s+\((\d+)%\)')

#Centros
df_final[['Total Centros', 'Centros Acertados']] = df_final['Centros (acertados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Centros'] = (
    df_final.apply(lambda row: (row['Centros Acertados'] / row['Total Centros'] * 100) if row['Total Centros'] != 0 else 0, axis=1).round(2)
)

#pases largos
df_final[['Total Pases Largos', 'Pases Largos Acertados']] = df_final['Pases largos (acertados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Pases Largos'] = (
    df_final.apply(lambda row: (row['Pases Largos Acertados'] / row['Total Pases Largos'] * 100) if row['Total Pases Largos'] != 0 else 0, axis=1).round(2)
)

#Duelos en el suelo
df_final[['Total Duelos el Suelo', 'Duelos Ganados en el Suelo']] = df_final['Duelos en el suelo (ganados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Duelos en el Suelo'] = (
    df_final.apply(lambda row: (row['Duelos Ganados en el Suelo'] / row['Total Duelos el Suelo'] * 100) if row['Total Duelos el Suelo'] != 0 else 0, axis=1).round(2)
)

#Duelos aéreos
df_final[['Total Duelos Aéreos', 'Duelos Ganados en el Aire']] = df_final['Duelos aéreos (ganados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Duelos Aéreos'] = (
    df_final.apply(lambda row: (row['Duelos Ganados en el Aire'] / row['Total Duelos Aéreos'] * 100) if row['Total Duelos Aéreos'] != 0 else 0, axis=1).round(2)
)

#Regates
df_final[['Total Regates', 'Regates Completados']] = df_final['Regates intentados (completados)'].str.extract(r'(\d+)\s+\((\d+)\)').apply(pd.to_numeric)
df_final['% Efectividad Regates'] = (
    df_final.apply(lambda row: (row['Regates Completados'] / row['Total Regates'] * 100) if row['Total Regates'] != 0 else 0, axis=1).round(2)
)



In [5]:
# Columnas tipo texto (al inicio)
columnas_texto = ['Nombre', 'Equipo', 'Posición']

# Columnas numéricas corregidas
columnas_numericas = [
    'Minutos jugados', 'Asistencias', 'Toques', 'Total Pases', 'Pases Acertados',
    'Efectividad Pases (%)', 'Pases clave', 'Total Centros', 'Centros Acertados',
    '% Efectividad Centros', 'Total Pases Largos', 'Pases Largos Acertados',
    '% Efectividad Pases Largos', 'Despejes', 'Tiros bloqueados', 'Intercepciones',
    'Tackles totales', 'Regateado', 'Total Duelos el Suelo', 'Duelos Ganados en el Suelo',
    '% Efectividad Duelos en el Suelo', 'Total Duelos Aéreos', 'Duelos Ganados en el Aire',
    '% Efectividad Duelos Aéreos', 'Faltas', 'Tiros a puerta', 'Tiros fuera',
    'Total Regates', 'Regates Completados', '% Efectividad Regates',
    'Goles', 'Posesiones perdidas', 'Ocas. claras creadas', 'Fueras de juego'
]

# Concatenar texto + numéricas ya convertidas
df_vf = pd.concat([
    df_final[columnas_texto],  # columnas de texto
    df_final[columnas_numericas].apply(pd.to_numeric, errors="coerce")  # columnas numéricas convertidas
], axis=1)

# Reemplazar NaN por 0
df_vf = df_vf.fillna(0)

In [6]:
ofensivas = [
    'Minutos jugados', 'Toques', 'Total Pases', 'Pases Acertados', 'Efectividad Pases (%)',
    'Asistencias', 'Pases clave', 'Total Centros', 'Centros Acertados',
    '% Efectividad Centros', 'Total Pases Largos', 'Pases Largos Acertados',
    '% Efectividad Pases Largos', 'Tiros a puerta', 'Tiros fuera',
    'Total Regates', 'Regates Completados', '% Efectividad Regates',
    'Goles', 'Ocas. claras creadas', 'Fueras de juego'
]

defensivas = [
    'Despejes', 'Tiros bloqueados', 'Intercepciones', 'Tackles totales',
    'Regateado', 'Total Duelos el Suelo', 'Duelos Ganados en el Suelo',
    '% Efectividad Duelos en el Suelo', 'Total Duelos Aéreos',
    'Duelos Ganados en el Aire', '% Efectividad Duelos Aéreos',
    'Faltas', 'Posesiones perdidas'
]

# 1) Limpiar y convertir a numérico
for col in ofensivas + defensivas:
    if col in df_final.columns:
        df_final[col] = (
            df_final[col]
            .astype(str)
            .str.replace(r'[%,]', '', regex=True)
        )
        df_final[col] = pd.to_numeric(df_final[col], errors='coerce').fillna(0)

# 2) Agrupar por equipo
ofensiva_por_equipo = df_final.groupby('Equipo')[ofensivas].sum().reset_index()
defensiva_por_equipo = df_final.groupby('Equipo')[defensivas].sum().reset_index()

# 3) Función para resaltar máximo y mínimo
def resaltar_max_min(s):
    if s.name == 'Equipo':
        return [''] * len(s)
    es_num = pd.to_numeric(s, errors='coerce')
    max_val = es_num.max()
    min_val = es_num.min()
    return [
        'background-color: green; color: black' if v == max_val
        else 'background-color: red; color: black' if v == min_val
        else 'color: black'
        for v in es_num
    ]

# Aplicar estilo con formato sin decimales
ofensiva_styled = (
    ofensiva_por_equipo
    .style
    .apply(resaltar_max_min, axis=0)
    .format(precision=0)
)

defensiva_styled = (
    defensiva_por_equipo
    .style
    .apply(resaltar_max_min, axis=0)
    .format(precision=0)
)

# <a id='toc5_'></a>[Teams Stats](#toc0_)

In [7]:
defensiva_styled

,Equipo,Despejes,Tiros bloqueados,Intercepciones,Tackles totales,Regateado,Total Duelos el Suelo,Duelos Ganados en el Suelo,% Efectividad Duelos en el Suelo,Total Duelos Aéreos,Duelos Ganados en el Aire,% Efectividad Duelos Aéreos,Faltas,Posesiones perdidas
0,Cerro Porteño,22,2,4,4,7,33,11,304,33,19,429,8,74
1,Estudiantes de La Plata,23,2,9,12,3,56,29,597,19,8,283,11,105


In [8]:
ofensiva_styled

,Equipo,Minutos jugados,Toques,Total Pases,Pases Acertados,Efectividad Pases (%),Asistencias,Pases clave,Total Centros,Centros Acertados,% Efectividad Centros,Total Pases Largos,Pases Largos Acertados,% Efectividad Pases Largos,Tiros a puerta,Tiros fuera,Total Regates,Regates Completados,% Efectividad Regates,Goles,Ocas. claras creadas,Fueras de juego
0,Cerro Porteño,651,270,158,103,697,0,3,8,2,120,43,15,365,1,3,6,1,33,0,0,3
1,Estudiantes de La Plata,856,605,433,360,899,2,14,9,5,250,56,25,449,10,3,14,6,200,3,2,1
